In [243]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    # Set of token names.   This is always required
    tokens = { ID, PRINT, BOOLEAN, EXTENDS, STRING, PUBLIC, LENGTH, STATIC, RETURN, WHILE, CLASS, 
              FALSE, ELSE, TRUE, VOID, MAIN, THIS, INT, NEW, IF, NUM, AND, NOT, EQUALS, LESS, PLUS,
              MINUS, TIMES, DOT, SEMICOLON, COMMA, LEFTBRACE, RIGHTBRACE, LEFTPARENT, RIGHTPARENT,
              LEFTSQRBRACKET, RIGHTSQRBRACKET}

    bad_tokens = [] 
    # String containing ignored characters
    ignore = ' \t\r\f'
    ignore_comment1 = r'\/\/.*'
    ignore_comment2 = r'\/\*.*\*\/'

    # Regular expression rules for tokens
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    EQUALS  = r'='
    PRINT   = r'System\.out\.println'
    DOT     = r'\.'
    LESS    = r'<'
    NOT     = r'!'
    AND     = r'&&'
    COMMA   = r','
    SEMICOLON       = r';'
    LEFTBRACE       = r'\{'
    RIGHTBRACE      = r'\}'
    LEFTPARENT      = r'\('
    RIGHTPARENT     = r'\)'
    LEFTSQRBRACKET  = r'\['
    RIGHTSQRBRACKET = r'\]'

    @_(r'\d+')
    def NUM(self, t):
        t.value = int(t.value)
        return t
    
    @_(r'true')
    def TRUE(self, t):
        t.value = True
        return t
    
    @_(r'false')
    def FALSE(self, t):
        t.value = False
        return t
    
    # Identifiers and keywords
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['if']    = IF
    ID['else']  = ELSE
    ID['while'] = WHILE
    ID['class'] = CLASS
    ID['public'] = PUBLIC
    ID['static'] = STATIC
    ID['void']   = VOID
    ID['main']   = MAIN
    ID['extends']= EXTENDS 
    ID['return'] = RETURN
    ID['int']    = INT
    ID['boolean']= BOOLEAN
    ID['new']    = NEW
    ID['length'] = LENGTH
    ID['this']   = THIS
    ID['String'] = STRING
    

    # Line number tracking
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print('Line %d: Bad character %r' % (self.lineno, t.value[0]))
        self.bad_tokens.append(t)
        self.index += 1

if __name__ == '__main__':
    data = '''
            class Factorial{
    public static void main(String[] a){
	System.out.println(new Fac().ComputeFac(10));
    }
}

class Fac {

    public int ComputeFac(int num){
	int num_aux ;
	if (num < 1)
	    num_aux = 1 ;
	else 
	    num_aux = num * (this.ComputeFac(num-1)) ;
	return num_aux ;
    }

}
        '''
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print(tok)

Token(type='CLASS', value='class', lineno=2, index=13)
Token(type='ID', value='Factorial', lineno=2, index=19)
Token(type='LEFTBRACE', value='{', lineno=2, index=28)
Token(type='PUBLIC', value='public', lineno=3, index=34)
Token(type='STATIC', value='static', lineno=3, index=41)
Token(type='VOID', value='void', lineno=3, index=48)
Token(type='MAIN', value='main', lineno=3, index=53)
Token(type='LEFTPARENT', value='(', lineno=3, index=57)
Token(type='STRING', value='String', lineno=3, index=58)
Token(type='LEFTSQRBRACKET', value='[', lineno=3, index=64)
Token(type='RIGHTSQRBRACKET', value=']', lineno=3, index=65)
Token(type='ID', value='a', lineno=3, index=67)
Token(type='RIGHTPARENT', value=')', lineno=3, index=68)
Token(type='LEFTBRACE', value='{', lineno=3, index=69)
Token(type='PRINT', value='System.out.println', lineno=4, index=72)
Token(type='LEFTPARENT', value='(', lineno=4, index=90)
Token(type='NEW', value='new', lineno=4, index=91)
Token(type='ID', value='Fac', lineno=4, index

In [244]:
from sly import Parser
#from calclex import CalcLexer

class CalcParser(Parser):
    # Get the token list from the lexer (required)
    tokens = CalcLexer.tokens

    debugfile = 'parser.out'

    # Grammar rules and actions

    precedence = (
        ('left', LESS),
        ('left', AND),
        ('right', NOT),
        ('left', PLUS, MINUS),
        ('left', TIMES),
    )
    
    # ------------------------------ PROGRAM ---------------------------------------
    @_('mainClass classAux')
    def program(self, p):
        return p
    
    @_('classDecl classAux')
    def classAux(self, p):
        return p
    
    @_('')
    def classAux(self, p):
        return ''
    
    
    # ------------------------------ MAIN CLASS ---------------------------------------

    @_('CLASS ID LEFTBRACE PUBLIC STATIC VOID MAIN LEFTPARENT'+
       ' STRING LEFTSQRBRACKET RIGHTSQRBRACKET ID RIGHTPARENT LEFTBRACE stm RIGHTBRACE RIGHTBRACE')
    def mainClass(self, p):
        return p


    # ------------------------------ CLASS Declaration ---------------------------------------
    @_('CLASS ID LEFTBRACE varCDAux methodCDAux RIGHTBRACE')
    def classDecl(self, p):
        return p
    
    @_('CLASS ID EXTENDS ID LEFTBRACE varCDAux methodCDAux RIGHTBRACE')
    def classDecl(self, p):
        return p

    @_('varDecl varCDAux')
    def varCDAux(self, p):
        return p

    @_('')
    def varCDAux(self, p):
        return p

    @_('methodDecl methodCDAux')
    def methodCDAux(self, p):
        return p

    @_('')
    def methodCDAux(self, p):
        return ''

        
    # ------------------------------ Var Declaration ---------------------------------------

    @_('tp ID SEMICOLON')
    def varDecl(self, p):
        return p

    # ------------------------------ Method Declaration ---------------------------------------
    @_('PUBLIC tp ID LEFTPARENT formalList RIGHTPARENT LEFTBRACE varMDAux stmMDAux RETURN expr SEMICOLON RIGHTBRACE')
    def methodDecl(self, p):
        return p

    @_('varDecl varMDAux')
    def varMDAux(self, p):
        return p

    @_('')
    def varMDAux(self, p):
        return ''

    @_('stm stmMDAux')
    def stmMDAux(self, p):
        return p

    @_('')
    def stmMDAux(self, p):
        return ''

    @_('tp ID formalAux')
    def formalList(self, p):
        return p
    
    @_('')
    def formalList(self, p):
        return ''
    
    @_('formalRest formalAux')
    def formalAux(self, p):
        return p

    @_('')
    def formalAux(self, p):
        return p

    @_('COMMA tp ID')
    def formalRest(self, p):
        return p
    

        
    # ------------------------------ Type ---------------------------------------
    @_('INT LEFTSQRBRACKET RIGHTSQRBRACKET')
    def tp(self, p):
        return p
    
    @_('BOOLEAN')
    def tp(self, p):
        return p
    
    @_('INT')
    def tp(self, p):
        return p
    
    @_('ID')
    def tp(self, p):
        return p

    # ------------------------------ STATEMENT ---------------------------------------
    @_('LEFTBRACE stmAux RIGHTBRACE')
    def stm(self, p):
        return p

    @_('stm stmAux')
    def stmAux(self, p):
        return p

    @_('')
    def stmAux(self, p):
        return p

    @_('stmTerm')
    def stm(self, p):
        return p

    @_('IF LEFTPARENT expr RIGHTPARENT stm ELSE stm')
    def stmTerm(self, p):
        return p
    
    @_('WHILE LEFTPARENT expr RIGHTPARENT stm')
    def stmTerm(self, p):
        return p
    
    @_('PRINT LEFTPARENT expr RIGHTPARENT SEMICOLON')
    def stmTerm(self, p):
        return p
    
    @_('ID EQUALS expr SEMICOLON')
    def stmTerm(self, p):
        return p
    
    @_('ID LEFTSQRBRACKET expr RIGHTSQRBRACKET EQUALS expr SEMICOLON')
    def stmTerm(self, p):
        return p


    # ------------------------------ EXPRESSÔES ---------------------------------------
    @_('expr AND oper')
    def expr(self, p):
        return p
    
    @_('expr LESS oper')
    def expr(self, p):
        return p
    
    @_('oper')
    def expr(self, p):
        return p.oper
    
    @_('oper PLUS term')
    def oper(self, p):
        return p

    @_('oper MINUS term')
    def oper(self, p):
        return p
    
    @_('term')
    def oper(self, p):
        return p.term

    @_('term TIMES factor')
    def term(self, p):
        return p

    @_('factor')
    def term(self, p):
        return p.factor

    @_('NUM')
    def factor(self, p):
        return p.NUM
    
    @_('TRUE')
    def expr(self, p):
        return p.TRUE
    
    @_('FALSE')
    def expr(self, p):
        return p.FALSE
    
    @_('THIS')
    def expr(self, p):
        return p.THIS
    
    @_('NOT expr')
    def expr(self, p):
        return p
    
    @_('LEFTPARENT expr RIGHTPARENT')
    def expr(self, p):
        return p
    
    @_('expr DOT LENGTH')
    def expr(self, p):
        return p
    
    @_('ID')
    def expr(self, p):
        return p
    
    @_('NEW ID LEFTPARENT RIGHTPARENT')
    def expr(self, p):
        return p
    
    @_('NEW INT LEFTSQRBRACKET expr RIGHTSQRBRACKET')
    def expr(self, p):
        return p
    
    @_('expr LEFTSQRBRACKET expr RIGHTSQRBRACKET')
    def expr(self, p):
        return p
    
    @_('expr DOT ID LEFTPARENT exprList RIGHTPARENT')
    def expr(self, p):
        return p
    
    @_('expr exprRest')
    def exprList(self, p):
        return p
    
    @_('')
    def exprList(self, p):
        return ''
    

    @_('COMMA expr exprRest')
    def exprRest(self, p):
        return p
    
    @_('')
    def exprRest(self, p):
        return ''
    # ------------------------------------------------


    def error(self, p):
        self.syntax_error = True


if __name__ == '__main__':
        lexer = CalcLexer()
        parser = CalcParser()

    #while True:
        try:
            text = '''
            teste && 5
        
            '''
            result = parser.parse(lexer.tokenize(text))
            print(result)
        except EOFError:
            ...
            #pass
            #break

None


Parser debugging for CalcParser written to parser.out


In [10]:
!pip3 install ply